In [2]:
import os
import sys

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

2022-05-02 20:23:12 Configured OSMnx 1.1.1
2022-05-02 20:23:12 HTTP response caching is on


In [3]:
df = pd.read_json("../scripts/areas.json")
mpos_folder = 'mpos_2020'
mun_gdf = gpd.GeoDataFrame()
marg_hex = gpd.GeoDataFrame()
prox_hex = gpd.GeoDataFrame()
pop_hex = gpd.GeoDataFrame()

In [4]:
cats = {'Alto/Muy Alto_Far','Alto/Muy Alto_Mid', 'Alto/Muy Alto_Near', 
'Bajo/Medio_Far', 'Bajo/Medio_Mid', 'Bajo/Medio_Near',
'Muy Bajo/Bajo_Far', 'Muy Bajo/Bajo_Mid', 'Muy Bajo/Bajo_Near' }
pop_cat = pd.DataFrame(index=(df.columns.unique()), columns =(cats))
pop_cat = pop_cat.fillna(0)


In [5]:
#options 'carne', 'aves', 'pescado', 'verdulerias', 'granos', 'lacteos'
food = 'verdulerias'

amenities = {'carne':[461121],'aves':[461122], 'pescados':[461123],
'verdulerias':[461130], 'granos':[461140], 'lacteos': [461150]}

In [6]:
for c in df.columns.unique():
    for i in range(len(df.loc["mpos", c])):
        # Extracts specific municipality code
        m = df.loc["mpos", c][i]
        # Downloads municipality polygon according to code
        query = f"SELECT * FROM marco.{mpos_folder} WHERE \"CVEGEO\" LIKE \'{m}\'"
        mun_gdf = mun_gdf.append(aup.gdf_from_query(query, geometry_col='geometry'))
        query = f"SELECT * FROM censo.hex_bins_marg_2020 WHERE \"CVEGEO\" LIKE \'{m}\'"
        marg_hex = marg_hex.append(aup.gdf_from_query(query, geometry_col='geometry'))
        query = f"SELECT * FROM censo.hex_bins_pop_2020 WHERE \"CVEGEO\" LIKE \'{m}\'"
        pop_hex = pop_hex.append(aup.gdf_from_query(query, geometry_col='geometry'))
        query = f"SELECT * FROM time_amenities.hex_bins_time_2021_food WHERE \"CVEGEO\" LIKE \'{m}\'"
        prox_hex = prox_hex.append(aup.gdf_from_query(query, geometry_col='geometry'))

In [7]:
    #Creates wkt for query
    # It will be used to download the POI
    gdf_tmp = mun_gdf.copy()
    gdf_tmp = gdf_tmp.to_crs("EPSG:6372")
    gdf_tmp = gdf_tmp.buffer(1).reset_index().rename(columns={0:'geometry'})
    gdf_tmp = gdf_tmp.to_crs("EPSG:4326")
    poly_wkt = gdf_tmp.dissolve().geometry.to_wkt()[0]
    aup.log("Created wkt based on dissolved polygon")
    denue = gpd.GeoDataFrame()
    for a in amenities:
        #creates empty gdf for the POIs
        #Based on the SCIAN code, the POIs will be downloaded from the DB
        for cod in amenities[a]:
            query = f"SELECT * FROM denue.denue_2021 WHERE (ST_Intersects(geometry, \'SRID=4326;{poly_wkt}\')) AND (\"codigo_act\" = {cod})"
            denue = denue.append(aup.gdf_from_query(query, geometry_col='geometry'))
        aup.log(f"Downloaded accumulated total of {len(denue)} {a} from database for {c}")

In [8]:
    prox_hex = prox_hex.set_crs("EPSG:4326")
    pop_hex = pop_hex.set_crs("EPSG:4326")
    marg_hex = marg_hex.set_crs("EPSG:4326")
    denue = denue.set_crs("EPSG:4326")

In [9]:
    prox = prox_hex[['hex_id_8', f'time_{food}']]
    pop = pop_hex[['hex_id_8', 'pobtot']]
    marg = marg_hex[['geometry','hex_id_8', 'imn_2020']]

In [10]:
    prox = prox.set_index('hex_id_8')
    pop = pop.set_index('hex_id_8')
    marg = marg.set_index('hex_id_8')

In [11]:
    hex_gdf = marg.merge(pop.merge(prox, left_index= True, right_index= True), left_index= True, right_index= True)

In [12]:
    #Assign Marginalization Category based on IMN
    for idx,row in hex_gdf.iterrows():
        imn = hex_gdf.loc[idx,'imn_2020']
        if imn >= 0.966338129860716:
            hex_gdf.at[idx, 'gm_2020'] = 'Muy Bajo'
        elif imn < 0.966338129860716 and imn >= 0.946436329971899:
            hex_gdf.at[idx, 'gm_2020'] = 'Bajo'
        elif imn < 0.946436329971899 and imn >= 0.926536329689967:
            hex_gdf.at[idx, 'gm_2020'] = 'Medio'
        elif imn < 0.926536329689967 and imn >= 0.899997041374425:
            hex_gdf.at[idx, 'gm_2020'] = 'Alto'
        elif imn < 0.899997041374425:
            hex_gdf.at[idx, 'gm_2020'] = 'Muy Alto'


In [13]:
    #Assign quantile category for marginalization based on IMN
    for idx,row in hex_gdf.iterrows():
        imn = hex_gdf.loc[idx,'imn_2020']
        if imn >= 0.954388:
            hex_gdf.at[idx, 'quantile'] = 'Muy Bajo/Bajo'
        elif imn < 0.954388 and imn >= 0.926536:
            hex_gdf.at[idx, 'quantile'] = 'Bajo/Medio'
        elif imn < 0.926536:
            hex_gdf.at[idx, 'quantile'] = 'Alto/Muy Alto'


In [14]:
    #Assign distance category based on travel time to each amenity
    for idx,row in hex_gdf.iterrows():
        t_food = hex_gdf.loc[idx,f'time_{food}']
        #### Check for meat shops
        if t_food <= 15:
            hex_gdf.at[idx, f'Q_{food}'] = 'Near'
        elif t_food > 15 and t_food <= 30:
            hex_gdf.at[idx, f'Q_{food}'] = 'Mid'
        elif t_food > 30:
            hex_gdf.at[idx, f'Q_{food}'] = 'Far'

In [15]:
    #Create Bivariate categories for all amenities
    hex_gdf[f'biv_{food}'] = hex_gdf['quantile'] + '_' + hex_gdf[f'Q_{food}']

In [16]:
    hex_gdf.head()

,geometry,imn_2020,pobtot,time_verdulerias,gm_2020,quantile,Q_verdulerias,biv_verdulerias
hex_id_8,,,,,,,,
8849abc9ebfffff,"POLYGON ((-103.61319 20.42802, -103.61772 20.4...",0.941744,273.168550,23.334919,Medio,Bajo/Medio,Mid,Bajo/Medio_Mid
8849abc9e3fffff,"POLYGON ((-103.61374 20.43321, -103.61319 20.4...",0.955670,98.333336,40.174722,Bajo,Muy Bajo/Bajo,Far,Muy Bajo/Bajo_Far
8849abc9e7fffff,"POLYGON ((-103.59961 20.43722, -103.60414 20.4...",0.952236,1247.971700,13.733920,Bajo,Bajo/Medio,Near,Bajo/Medio_Near
8849abc9e1fffff,"POLYGON ((-103.60249 20.41857, -103.59796 20.4...",0.949248,2893.067000,9.851744,Bajo,Bajo/Medio,Near,Bajo/Medio_Near
8849abc9e9fffff,"POLYGON ((-103.60647 20.41031, -103.60194 20.4...",0.941744,1707.303600,7.181533,Medio,Bajo/Medio,Near,Bajo/Medio_Near


In [17]:
    #Create a gdf with simplified variables
    biv_gdf = hex_gdf[['geometry', 'pobtot', f'biv_{food}']]

In [18]:
    for idx,row in biv_gdf.iterrows():
        if biv_gdf.loc[idx,f'biv_{food}'] == 'Muy Bajo/Bajo_Mid':
            pop_cat.at[c, 'Muy Bajo/Bajo_Mid'] = pop_cat.loc[c, 'Muy Bajo/Bajo_Mid'] + biv_gdf.loc[idx,'pobtot']
        elif biv_gdf.loc[idx,f'biv_{food}'] == 'Alto/Muy Alto_Near':
            pop_cat.at[c, 'Alto/Muy Alto_Near'] = pop_cat.loc[c, 'Alto/Muy Alto_Near'] + biv_gdf.loc[idx,'pobtot']
        elif biv_gdf.loc[idx,f'biv_{food}'] == 'Alto/Muy Alto_Far':
            pop_cat.at[c, 'Alto/Muy Alto_Far'] = pop_cat.loc[c, 'Alto/Muy Alto_Far'] + biv_gdf.loc[idx,'pobtot']
        elif biv_gdf.loc[idx,f'biv_{food}'] == 'Muy Bajo/Bajo_Far':
            pop_cat.at[c, 'Muy Bajo/Bajo_Far'] = pop_cat.loc[c, 'Muy Bajo/Bajo_Far'] + biv_gdf.loc[idx,'pobtot']
        elif biv_gdf.loc[idx,f'biv_{food}'] == 'Alto/Muy Alto_Mid':
            pop_cat.at[c, 'Alto/Muy Alto_Mid'] = pop_cat.loc[c, 'Alto/Muy Alto_Mid'] + biv_gdf.loc[idx,'pobtot']
        elif biv_gdf.loc[idx,f'biv_{food}'] == 'Bajo/Medio_Near':
            pop_cat.at[c, 'Bajo/Medio_Near'] = pop_cat.loc[c, 'Bajo/Medio_Near'] + biv_gdf.loc[idx,'pobtot']
        elif biv_gdf.loc[idx,f'biv_{food}'] == 'Muy Bajo/Bajo_Near':
            pop_cat.at[c, 'Muy Bajo/Bajo_Near'] = pop_cat.loc[c, 'Muy Bajo/Bajo_Near'] + biv_gdf.loc[idx,'pobtot']
        elif biv_gdf.loc[idx,f'biv_{food}'] == 'Bajo/Medio_Far':
            pop_cat.at[c, 'Bajo/Medio_Far'] = pop_cat.loc[c, 'Bajo/Medio_Far'] + biv_gdf.loc[idx,'pobtot']
        elif biv_gdf.loc[idx,f'biv_{food}'] == 'Bajo/Medio_Mid':
            pop_cat.at[c, 'Bajo/Medio_Mid'] = pop_cat.loc[c, 'Bajo/Medio_Mid'] + biv_gdf.loc[idx,'pobtot']

In [19]:
    hex = hex_gdf[['geometry']].copy()
    # Perform spatial join to match points and polygons
    pointInPolys = gpd.tools.sjoin(denue, hex, op="within", how='left')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [20]:
table = pd.crosstab(pointInPolys.index_right, pointInPolys.codigo_act )

In [21]:
hex = hex.merge(table, left_index=True, right_index=True)

In [22]:
hex_capita = hex.merge(pop, right_index=True, left_index=True)

In [23]:
hex_capita

,geometry,461121,461122,461123,461130,461140,461150,pobtot
8849abc9e7fffff,"POLYGON ((-103.59961 20.43722, -103.60414 20.4...",2,1,0,2,0,0,1247.9717
8849abc9e1fffff,"POLYGON ((-103.60249 20.41857, -103.59796 20.4...",1,0,0,0,0,0,2893.0670
8849abc9e9fffff,"POLYGON ((-103.60647 20.41031, -103.60194 20.4...",2,3,0,1,0,0,1707.3036
8849abc933fffff,"POLYGON ((-103.58274 20.41526, -103.58329 20.4...",1,2,0,1,0,0,761.9108
8849abc9d7fffff,"POLYGON ((-103.64365 20.44077, -103.64818 20.4...",0,2,1,0,2,0,2816.9734
...,...,...,...,...,...,...,...,...
8849aa64d9fffff,"POLYGON ((-103.08035 20.52653, -103.08088 20.5...",4,1,1,1,0,0,1645.4884
88498c9a11fffff,"POLYGON ((-103.04584 20.61630, -103.04637 20.6...",1,0,0,0,0,0,1191.7736
88498c9123fffff,"POLYGON ((-103.07920 20.61053, -103.07466 20.6...",1,0,0,0,0,1,1761.0980
88498c9159fffff,"POLYGON ((-103.11952 20.57787, -103.12005 20.5...",3,1,1,1,0,0,1421.9443


In [24]:
hex_capita[f'carn_t'] = hex_capita[461121]
hex_capita[f'aves_t'] = hex_capita[461122]
hex_capita[f'pesca_t'] = hex_capita[461123]
hex_capita[f'verd_t'] = hex_capita[461130]
hex_capita[f'grano_t'] = hex_capita[461140]
hex_capita[f'lact_t'] = hex_capita[461150]

In [25]:
hex_capita.head()

,geometry,461121,461122,461123,461130,461140,461150,pobtot,carn_t,aves_t,pesca_t,verd_t,grano_t,lact_t
8849abc9e7fffff,"POLYGON ((-103.59961 20.43722, -103.60414 20.4...",2,1,0,2,0,0,1247.9717,2,1,0,2,0,0
8849abc9e1fffff,"POLYGON ((-103.60249 20.41857, -103.59796 20.4...",1,0,0,0,0,0,2893.0670,1,0,0,0,0,0
8849abc9e9fffff,"POLYGON ((-103.60647 20.41031, -103.60194 20.4...",2,3,0,1,0,0,1707.3036,2,3,0,1,0,0
8849abc933fffff,"POLYGON ((-103.58274 20.41526, -103.58329 20.4...",1,2,0,1,0,0,761.9108,1,2,0,1,0,0
8849abc9d7fffff,"POLYGON ((-103.64365 20.44077, -103.64818 20.4...",0,2,1,0,2,0,2816.9734,0,2,1,0,2,0


In [26]:
pt = hex_capita[['geometry', 'carn_t', 'aves_t', 'pesca_t', 'verd_t', 'grano_t', 'lact_t']]

In [27]:
pt.to_file('food_tot_GDL.geojson')